## Transcripción de voz a texto en tiempo real

In [2]:
# python version == 3.8.20

!pip install pyaudio
!pip install sounddevice
!pip install torch
!pip install webrtcvad
!pip install numpy

In [4]:
import sounddevice as sd
import numpy as np
import whisper
import queue
import pyaudio
import wave
import webrtcvad

# Configuración de PyAudio
#CHUNK = 4096    #1024              # Tamaño de cada fragmento de audio
FORMAT = pyaudio.paInt16  # Formato de audio
CHANNELS = 1              # Audio mono
RATE = 16000 #11025              # Frecuencia de muestreo compatible con Whisper
DURATION = 0.03
BYTES_PRE_SAMPLE = 1
chunk = DURATION * RATE  
chunk = int(chunk)
# Inicializar PyAudio
audio = pyaudio.PyAudio()
# Inicializar cola para audio
audio_queue = queue.Queue()

# Crear el modelo de Whisper
model = whisper.load_model("base")  # Cambiar el modelo según sea necesario

# Inicializar VAD
vad = webrtcvad.Vad()
vad.set_mode(3)  # Nivel de sensibilidad: 0 (menos sensible) a 3 (más sensible)

# Función para detectar actividad de voz
def is_speech(data, sample_rate):
#    print("checking if is speech...")
    return vad.is_speech(data, sample_rate)

# Función para reproducir audio
def play_audio(data):
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True)
    stream.write(data)
    stream.stop_stream()
    stream.close()

# Función para capturar audio en tiempo real
def audio_callback(indata, frames, time, status):
    if status:
        print(f"Status: {status}")
    audio_queue.put(indata.copy())

print("Dispositivos de audio disponibles:")
for i in range(audio.get_device_count()):
    info = audio.get_device_info_by_index(i)
    print(f"ID: {i}, Nombre: {info['name']}, Entrada: {info['maxInputChannels']}, Salida: {info['maxOutputChannels']}")


c:\ProgramData\anaconda3\envs\melo\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Dispositivos de audio disponibles:
ID: 0, Nombre: Asignador de sonido Microsoft - Input, Entrada: 2, Salida: 0
ID: 1, Nombre: Microphone (Realtek(R) Audio), Entrada: 2, Salida: 0
ID: 2, Nombre: Asignador de sonido Microsoft - Output, Entrada: 0, Salida: 2
ID: 3, Nombre: Headphones (Realtek(R) Audio), Entrada: 0, Salida: 2
ID: 4, Nombre: Acer V246HQL (HD Audio Driver f, Entrada: 0, Salida: 2
ID: 5, Nombre: Speakers (Realtek(R) Audio), Entrada: 0, Salida: 2
ID: 6, Nombre: Controlador primario de captura de sonido, Entrada: 2, Salida: 0
ID: 7, Nombre: Microphone (Realtek(R) Audio), Entrada: 2, Salida: 0
ID: 8, Nombre: Controlador primario de sonido, Entrada: 0, Salida: 2
ID: 9, Nombre: Headphones (Realtek(R) Audio), Entrada: 0, Salida: 2
ID: 10, Nombre: Acer V246HQL (HD Audio Driver for Display Audio), Entrada: 0, Salida: 2
ID: 11, Nombre: Speakers (Realtek(R) Audio), Entrada: 0, Salida: 2
ID: 12, Nombre: Headphones (Realtek(R) Audio), Entrada: 0, Salida: 2
ID: 13, Nombre: Acer V246HQL (H

In [2]:
# Opcion 1.- Real-Time con sounddevice
with sd.InputStream(samplerate=RATE, channels=CHANNELS, callback=audio_callback):
    print("Grabando... Presiona Ctrl+C para detener.")
    try:
        # Acumular audio y transcribir en tiempo real
        while True:
            audio_data = audio_queue.get()
            # Convertir audio a un formato que Whisper entienda
            audio_np = np.squeeze(audio_data)
            result = model.transcribe(audio_np, language="es")
            print("Transcripción: ", result["text"])
    except KeyboardInterrupt:
        print("Grabación detenida.")
    except Exception as e:
        print(f"Error: {e}")


Grabando... Presiona Ctrl+C para detener.


c:\ProgramData\anaconda3\envs\whisper\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción:  
Transcripción:  
Transcripción:  
Transcripción:  
Transcripción:  
Transcripción:  
Transcripción:  
Transcripción:  
Grabación detenida.


In [7]:
# Opcion 2.- Real-Time con pyaudio
# transcripción contínua periódica
# Abrir el stream para capturar el audio del micrófono
stream = audio.open(format=FORMAT, 
                    channels=CHANNELS, 
                    rate=RATE, 
                    input=True, 
                    frames_per_buffer=chunk)
print("Grabando... Presiona Ctrl+C para detener.")

try:
    audio_buffer = b''  # Buffer para almacenar audio capturado
    while True:
        # Leer audio desde el micrófono
        data = stream.read(chunk, exception_on_overflow=False)
        audio_buffer += data

        # Procesar el audio cada 5 segundos aproximadamente
        if len(audio_buffer) >= RATE * 5:
            # Convertir a formato que Whisper entienda
            audio_np = np.frombuffer(audio_buffer, dtype=np.int16).astype(np.float32) / 32768.0

            # Transcribir el audio
            result = model.transcribe(audio_np, language="es")
            print("Transcripción: ", result["text"])

            # Reproducir el audio capturado
            play_audio(audio_buffer)

            # Limpiar el buffer después de procesar
            audio_buffer = b''

except KeyboardInterrupt:
    print("Grabación detenida.")
except Exception as e:
    print(f"Error: {e}")

# Cerrar streams y PyAudio
stream.stop_stream()
stream.close()
audio.terminate()


Grabando... Presiona Ctrl+C para detener.
Transcripción:   esto es porque se queda abierto
Transcripción:  
Grabación detenida.


In [5]:
# opcion 3.- transcripción mediante detección activa de voz VAD

INPUT_DEVICE_ID = 1  # Cambiar según el dispositivo de entrada
OUTPUT_DEVICE_ID = 3  # Cambiar según el dispositivo de salida

# Abrir el stream para capturar el audio del micrófono
stream = audio.open(format=FORMAT, 
                    channels=CHANNELS, 
                    rate=RATE, 
                    input=True, 
#                    output=False,
#                    input_device_index=INPUT_DEVICE_ID,
#                    output_device_index=OUTPUT_DEVICE_ID,
                    frames_per_buffer=chunk)

print("Esperando actividad de voz... Presiona Ctrl+C para detener.")

try:
    audio_buffer = b''  # Buffer para almacenar audio capturado
    speaking = False    # Indicador de actividad de voz
    while True:
        # Leer audio desde el micrófono
        data = stream.read(chunk, exception_on_overflow=False)
        
        # Detectar actividad de voz
        if is_speech(data, RATE):
            if not speaking:
                print("Usuario comenzó a hablar...")
                speaking = True
            audio_buffer += data
        else:
            if speaking:
                print("Usuario terminó de hablar.")
                speaking = False
                
                # Convertir el audio capturado a formato que Whisper entienda
                audio_np = np.frombuffer(audio_buffer, dtype=np.int16).astype(np.float32) / 32768.0
                
                # Transcribir el audio
                print("Transcribiendo...")
                result = model.transcribe(audio_np, language="es")
                print("Transcripción: ", result["text"])
                
                # Limpiar el buffer después de procesar
                audio_buffer = b''

except KeyboardInterrupt:
    print("Grabación detenida.")
except Exception as e:
    print(f"Error: {e}")

# Cerrar streams y PyAudio
stream.stop_stream()
stream.close()
audio.terminate()


Esperando actividad de voz... Presiona Ctrl+C para detener.
Usuario comenzó a hablar...
Usuario terminó de hablar.
Transcribiendo...


c:\ProgramData\anaconda3\envs\melo\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción:   Hola, esto es un texto de prueba.
Usuario comenzó a hablar...
Usuario terminó de hablar.
Transcribiendo...
Transcripción:   Parece que funciona bien, ¿no?
Grabación detenida.
